# Generate word2vec model - include all new topic words

In [1]:
from gensim.models import Word2Vec, KeyedVectors
import gensim
import time
import numpy as np
from tqdm import tqdm
from pprint import pprint
import pickle
import spacy
import re
import operator
import spacy_udpipe

In [2]:
nlp = spacy.load('fi_experimental_web_md')

In [3]:
# set variables 
LANG = 'fi'
MAX_WORDS = 500000  # maximum processed words to small model 

MODEL_DIR = '/mnt/data/data/models-fasttext/'
TOPICS_DIR = '/home/dzon/kajo/topics/topic_lists-based_on_en-US/'

model_filename = MODEL_DIR + 'cc.'+ LANG +'.300-lemma_500k_min2.vec.bin'

excluded_model_name = MODEL_DIR + 'cc.'+ LANG +'.300-lemma_500k_min2_excluded.vec.bin'

model_orig_filename = MODEL_DIR + 'cc.'+ LANG +'.300.vec.bin'
model_orig_filename_vec = MODEL_DIR + 'cc.'+ LANG +'.300.vec.gz'

pickled_words_lemmas = MODEL_DIR + LANG + '_ft_all_words_lemmas.pickle'

lemma_list_file = TOPICS_DIR + 'topics_lemma_'+ LANG +'.lst'
nolemma_list_file = TOPICS_DIR + 'topics_'+ LANG +'.lst'

excluded_filename = TOPICS_DIR + 'topics_' + LANG + '-not_in_model.lst'

lemma_list_file_excluded = TOPICS_DIR + 'topics_lemma_excluded_' + LANG + '.lst'
nolemma_list_file_excluded = TOPICS_DIR + 'topics_excluded_' + LANG + '.lst'


In [4]:
# load the big model
model_big = KeyedVectors.load_word2vec_format(model_orig_filename, binary=True)

In [5]:
# process all words in fasttext model  takes ~1.5h for FI
all_words_lemmas = []

lemmas = {}
word_lemma = {}
words = model_big.index2word
for word in tqdm(words):
    # skip "words" which are not regular words
    if not re.fullmatch("([^\d\W])+", word):
        continue
    doc = nlp(word)
    lemma = doc[0].lemma_.lower()
    all_words_lemmas.append((word, lemma))
print("Number of words in big model: ", len(all_words_lemmas))

#save pickled dump 
with open(pickled_words_lemmas, 'wb') as f:
    pickle.dump(all_words_lemmas, f)

100%|██████████| 2000000/2000000 [2:42:02<00:00, 205.71it/s]  


Number of words in big model:  1746550


In [6]:
#load pickled lemma list
with open(pickled_words_lemmas, 'rb') as f:
    all_words_lemmas = pickle.load(f)
pprint(all_words_lemmas[:50])

[('ja', 'ja'),
 ('on', 'olla'),
 ('oli', 'olla'),
 ('ei', 'ei'),
 ('että', 'että'),
 ('myös', 'myös'),
 ('tai', 'tai'),
 ('Lisätiedot', 'lisätieto'),
 ('ovat', 'olla'),
 ('mutta', 'mutta'),
 ('joka', 'joka'),
 ('se', 'se'),
 ('sekä', 'sekä'),
 ('n', 'n'),
 ('vuonna', 'vuosi'),
 ('kun', 'kun'),
 ('hän', 'hän'),
 ('sen', 'se'),
 ('ole', 'olla'),
 ('kuin', 'kuin'),
 ('niin', 'niin'),
 ('kanssa', 'kanssa'),
 ('voi', 'voi'),
 ('kello', 'kello'),
 ('Hän', 'hän'),
 ('mukaan', 'mukaan'),
 ('jälkeen', 'jälkeen'),
 ('klo', 'klo'),
 ('jo', 'jo'),
 ('Suomen', 'suomi'),
 ('vain', 'vain'),
 ('s', 's'),
 ('ollut', 'olla'),
 ('olla', 'olla'),
 ('p', 'p'),
 ('vuoden', 'vuosi'),
 ('jossa', 'joka'),
 ('sitä', 'se'),
 ('jonka', 'joka'),
 ('_', '_'),
 ('jotka', 'joka'),
 ('kuitenkin', 'kuitenkin'),
 ('vielä', 'vielä'),
 ('jos', 'jos'),
 ('nyt', 'nyt'),
 ('eli', 'eli'),
 ('of', 'of'),
 ('sitten', 'sitten'),
 ('noin', 'noin'),
 ('Se', 'se')]


In [7]:
# generate helpful dictionaries/lists
big_lemmas_set = set([l for (_,l) in all_words_lemmas])
print(len(big_lemmas_set))

big_words_set = set([w for (w,_) in all_words_lemmas])
print(len(big_words_set))

lemmas = {}
words_lower_orig = {}
big_lemma_word_dic = {}
big_word_lemma_dic = {}
for (w,l) in all_words_lemmas:
    big_word_lemma_dic[w] = l
    
    #dictionary of lowercase word -> all original words
    if w.lower() in words_lower_orig:
        words_lower_orig[w.lower()].append(w)
    else:
        words_lower_orig[w.lower()] = [w]
        
    if l in big_lemma_word_dic:
        big_lemma_word_dic[l].append(w)
        lemmas[l] += 1
    else:
        big_lemma_word_dic[l] = [w]
        lemmas[l] = 1
        
# load all the words from big model
words = model_big.wv.index2word
words_lower = [w.lower() for w in words]

#words_lemma = model_lemma_big.index2word       

746051
1746550


/home/dzon/kajo/kajo_semexp_core/lib/python3.6/site-packages/ipykernel_launcher.py:29: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


In [8]:
# sort the lemmas from most frequent
sorted_lemmas = sorted(lemmas.items(), key=operator.itemgetter(1), reverse=True)
# trying first words
sorted_lemmas[:20]

[('olla', 624),
 ('saada', 473),
 ('tehdä', 439),
 ('käyttää', 377),
 ('antaa', 336),
 ('esittää', 327),
 ('tulla', 311),
 ('hyvä', 310),
 ('pitää', 309),
 ('valmistaa', 304),
 ('ottaa', 299),
 ('valita', 286),
 ('tietää', 284),
 ('nähdä', 283),
 ('haluta', 280),
 ('lukea', 273),
 ('mennä', 269),
 ('ostaa', 268),
 ('kirjoittaa', 264),
 ('katsoa', 261)]

In [9]:
# create vectors for the small models - average vectors for lemmas
# process only first MAX_WORDS in the big model
tmp_words = [None] * len(model_big.wv.index2word)
tmp_vecs = [None] * len(model_big.wv.index2word) 
for i,(word,lemma) in enumerate(all_words_lemmas[:MAX_WORDS]):
    lemma_words_list = big_lemma_word_dic[lemma]
    # add only lemmas made from at least 2 words
    if len(lemma_words_list)<2:
        continue
    tmp_vectors = []
    tmp_indexes = []
    error_with_word = False
    for word2 in lemma_words_list:
        try:
            tmp_vectors.append(model_big[word2])  
            tmp_indexes.append(model_big.vocab[word2].index)
        except:
            print(i, "error with ", word, " lemma ", lemma, "words", lemma_words_list)
            error_with_word = True
            continue
    if not error_with_word:
        try:
            tmp_words[min(tmp_indexes)] = lemma
            tmp_vecs[min(tmp_indexes)] = list(np.average(tmp_vectors, axis=0))
        except:
            print(i, "error with lemma ", lemma, " - words", lemma_words_list, " indexes: ", tmp_indexes)
            # note - we are processing first MAX_WORDS from the model after the filter out of the "non-words"
            # so some of the last words in that range may refer to words on bigger index than MAX_WORDS

# filter the lists (some positions remained None)
final_words = [i for i in tmp_words if i] 
final_vecs = [i for i in tmp_vecs if i != None] 

print(len(final_vecs), len(final_words))
len(final_vecs[0])  

del tmp_vecs
del tmp_words

/home/dzon/kajo/kajo_semexp_core/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  This is separate from the ipykernel package so we can avoid doing imports until
/home/dzon/kajo/kajo_semexp_core/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  after removing the cwd from sys.path.


108350 108350


In [10]:
# create new gensim word2vec model with the vectors:
new_vector_space = gensim.models.keyedvectors.Word2VecKeyedVectors(300)
new_vector_space.add(final_words,final_vecs)

In [14]:
# test the model
pprint(new_vector_space.most_similar('isä'))
pprint(new_vector_space.most_similar_cosmul(positive=['nainen', 'kuningas'], negative=['mies']))

[('äiti', 0.8811246752738953),
 ('poika', 0.852703332901001),
 ('veli', 0.8099314570426941),
 ('tytär', 0.8042938709259033),
 ('lapsi', 0.7681261897087097),
 ('ukki', 0.7546637654304504),
 ('vanhempi', 0.7475104331970215),
 ('isoisä', 0.7442623376846313),
 ('sisko', 0.7412679195404053),
 ('iskä', 0.7375667095184326)]
[('kuningatar', 0.8673739433288574),
 ('kuninkaallinen', 0.8453649878501892),
 ('kuningataräiti', 0.843066930770874),
 ('hallitsija', 0.8396105170249939),
 ('kuninkuus', 0.8304585814476013),
 ('kuninkaanvalta', 0.825010359287262),
 ('ruhtinas', 0.8243961930274963),
 ('kuninkaanlinna', 0.8229613304138184),
 ('kuningaspari', 0.8216860890388489),
 ('suurkuningas', 0.8214312195777893)]


In [15]:
# save the model
new_vector_space.save_word2vec_format(model_filename, binary=True)

In [16]:
#model = KeyedVectors.load_word2vec_format(model_filename, binary=True)
model = new_vector_space
words_model = model.wv.index2word

/home/dzon/kajo/kajo_semexp_core/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [17]:
# load topic listst
with open(nolemma_list_file) as fp:
    topics_nolemma = fp.readlines() 
print(len(topics_nolemma))
    
# load the lemma list
# with open(lemma_list_file) as fp:
#    topics = fp.readlines()
# print(len(topics))

topics = []
# generate the lemma list
for topic in tqdm(topics_nolemma):
    topic = re.sub('\W',' ',topic)
    topic = re.sub('\ +',' ',topic)
    doc = nlp(topic.strip().lower())
    lemmas = []
        
    for token in doc:
        if token.is_stop:
            continue
        lemmas.append(token.lemma_)    
    if len(lemmas) == 0:
        print('lemmas = 0 ', topics_nolemma)
    topics.append("_".join(lemmas))
    
print(len(topics))    
    
#write the lemma topic list
with open(lemma_list_file, 'wt', encoding='utf8') as f:
    for topic_lemma in topics:
        f.write(topic_lemma + "\n")    

  0%|          | 1/5305 [00:00<12:20,  7.17it/s]

5305


 78%|███████▊  | 4163/5305 [00:16<00:03, 288.24it/s]

lemmas = 0  ['paikallinen kehitys\n', 'maaseuturakentaminen\n', 'vesitalous\n', 'koulutustaso\n', 'yleissivistävä koulutus\n', 'rantapenger\n', 'yhteisöoikeus\n', 'kulttuuriperintö\n', 'viljelysmaa, maatalousmaa\n', 'maatalousmaisema\n', 'maataloustuotanto\n', 'taloudellinen kehitys\n', 'ravintoketju\n', 'elintarviketeollisuus\n', 'ruuan laatu\n', 'globaali lämpeneminen\n', 'kasvihuoneilmiö\n', 'elinkeinoelämän kehitys\n', 'työmarkkinat\n', 'pääsy maa-alueelle\n', 'uusjako, tilusten sijoitus\n', 'maankäytön suunnittelu\n', 'maisema-arkkitehtuuri\n', 'maisemansuunnittelu\n', 'maankäyttö\n', 'kaavoitus, maankäytön suunnittelu\n', 'meriympäristö\n', 'luonnonsuojelu\n', 'luonnonsuojelu\n', 'biodynaaminen viljely\n', 'fyysinen suunnittelu\n', 'alkeisopetus, perusasteen koulutus\n', 'autoteollisuus\n', 'elämänlaatu\n', 'aluekehitys\n', 'resurssijohtaminen\n', 'maaseutuväestö\n', 'keskiasteen koulutus\n', 'maan tiivistys\n', 'maan kasvukunnon heikkeneminen\n', 'maaeroosio\n', 'kestävä kehitys

 81%|████████  | 4305/5305 [00:16<00:03, 267.77it/s]

lemmas = 0  ['paikallinen kehitys\n', 'maaseuturakentaminen\n', 'vesitalous\n', 'koulutustaso\n', 'yleissivistävä koulutus\n', 'rantapenger\n', 'yhteisöoikeus\n', 'kulttuuriperintö\n', 'viljelysmaa, maatalousmaa\n', 'maatalousmaisema\n', 'maataloustuotanto\n', 'taloudellinen kehitys\n', 'ravintoketju\n', 'elintarviketeollisuus\n', 'ruuan laatu\n', 'globaali lämpeneminen\n', 'kasvihuoneilmiö\n', 'elinkeinoelämän kehitys\n', 'työmarkkinat\n', 'pääsy maa-alueelle\n', 'uusjako, tilusten sijoitus\n', 'maankäytön suunnittelu\n', 'maisema-arkkitehtuuri\n', 'maisemansuunnittelu\n', 'maankäyttö\n', 'kaavoitus, maankäytön suunnittelu\n', 'meriympäristö\n', 'luonnonsuojelu\n', 'luonnonsuojelu\n', 'biodynaaminen viljely\n', 'fyysinen suunnittelu\n', 'alkeisopetus, perusasteen koulutus\n', 'autoteollisuus\n', 'elämänlaatu\n', 'aluekehitys\n', 'resurssijohtaminen\n', 'maaseutuväestö\n', 'keskiasteen koulutus\n', 'maan tiivistys\n', 'maan kasvukunnon heikkeneminen\n', 'maaeroosio\n', 'kestävä kehitys

 87%|████████▋ | 4624/5305 [00:17<00:02, 285.62it/s]

lemmas = 0  ['paikallinen kehitys\n', 'maaseuturakentaminen\n', 'vesitalous\n', 'koulutustaso\n', 'yleissivistävä koulutus\n', 'rantapenger\n', 'yhteisöoikeus\n', 'kulttuuriperintö\n', 'viljelysmaa, maatalousmaa\n', 'maatalousmaisema\n', 'maataloustuotanto\n', 'taloudellinen kehitys\n', 'ravintoketju\n', 'elintarviketeollisuus\n', 'ruuan laatu\n', 'globaali lämpeneminen\n', 'kasvihuoneilmiö\n', 'elinkeinoelämän kehitys\n', 'työmarkkinat\n', 'pääsy maa-alueelle\n', 'uusjako, tilusten sijoitus\n', 'maankäytön suunnittelu\n', 'maisema-arkkitehtuuri\n', 'maisemansuunnittelu\n', 'maankäyttö\n', 'kaavoitus, maankäytön suunnittelu\n', 'meriympäristö\n', 'luonnonsuojelu\n', 'luonnonsuojelu\n', 'biodynaaminen viljely\n', 'fyysinen suunnittelu\n', 'alkeisopetus, perusasteen koulutus\n', 'autoteollisuus\n', 'elämänlaatu\n', 'aluekehitys\n', 'resurssijohtaminen\n', 'maaseutuväestö\n', 'keskiasteen koulutus\n', 'maan tiivistys\n', 'maan kasvukunnon heikkeneminen\n', 'maaeroosio\n', 'kestävä kehitys

100%|██████████| 5305/5305 [00:20<00:00, 261.38it/s]

5305


In [18]:
print(len(topics),len(topics_nolemma))  

5305 5305


In [19]:
#try if the topic-lemma words exist in the models  

words_to_manual_add = {}

for i, (topic, topic_nolemma) in enumerate(zip(topics, topics_nolemma)):
    # process the nolemma topic so we can see which word was lemmatized to which lemma
    topic_nolemma = re.sub('\W',' ',topic_nolemma)
    topic_nolemma = re.sub('\ +',' ',topic_nolemma)
    doc = nlp(topic_nolemma.strip().lower())
    nolemmas = []
    for token in doc:
        if token.is_stop:
            continue
        else:
            nolemmas.append(token.text)  
    
    topic = topic.strip()
    topic_words = topic.split('_')
    if len(topic_words) != len(nolemmas):
        print("!!!!! error - not equal length: ", nolemmas, topic_words)
        continue
    for word, word_nolemma in zip(topic_words, nolemmas):
        #print(word)
        found = 0
        if word in words_model:
            found = 1
            #print("l1")
        elif word in big_lemmas_set:
            found = 1
            print("l2", word)
        elif word in words_lower:
            found = 1
            print("l3", word)
        # nolemma words are in PL case probably not needed and all lemmas exist in lemma list
        elif word_nolemma in words_model:
            found = 1
            print("w1", word)
        elif word_nolemma in big_lemmas_set:
            found = 1
            print("w2", word)
        elif word_nolemma in words_lower:
            found = 1
            print("w3", word)

        if found == 0:
            print(i, 'needs to be added manually: ', word, " - ", word_nolemma, " - ", topic_nolemma.strip())
            if word not in words_to_manual_add:
                words_to_manual_add[word] = topic_nolemma.strip()
                    
print(len(words_to_manual_add))          

l2 rantapenger
l2 yhteisöoikeus
l2 maatalousmaisema
23 needs to be added manually:  maisemansuunnittelu  -  maisemansuunnittelu  -  maisemansuunnittelu
35 needs to be added manually:  resurssijohtaminen  -  resurssijohtaminen  -  resurssijohtaminen
40 needs to be added manually:  maaeroosio  -  maaeroosio  -  maaeroosio
l2 vesieroosio
l2 tuulieroosio
l2 ympäristömuutos
63 needs to be added manually:  moottoriajoneuvoteollisuus  -  moottoriajoneuvoteollisuus  -  moottoriajoneuvoteollisuus
64 needs to be added manually:  maaseutusuunnittelu  -  maaseutusuunnittelu  -  maaseutusuunnittelu
l2 luonnonsuojelujärjestö
76 needs to be added manually:  ruuanjalostusteollisuus  -  ruuanjalostusteollisuus  -  ruuanjalostusteollisuus
83 needs to be added manually:  luonnonsuojelupolitiikka  -  luonnonsuojelupolitiikka  -  luonnonsuojelupolitiikka
w2 Eurooppa
l2 luonnonsuojeluohjelma
l2 luonnonsuojelulainsäädäntö
96 needs to be added manually:  tuotantokeino  -  tuotantokeinot  -  maatalouden tuotan

630 needs to be added manually:  ympäristönseuranta  -  ympäristönseuranta  -  ympäristönseuranta
635 needs to be added manually:  johtamissuunnitelma  -  johtamissuunnitelma  -  hallintasuunnitelma johtamissuunnitelma
l2 selluloosateollisuus
l2 finanssioppi
654 needs to be added manually:  maanviljelyskoneisto  -  maanviljelyskoneisto  -  maanviljelyskoneisto
l2 terveydenhuoltolainsäädäntö
661 needs to be added manually:  teollisuustaloustiede  -  teollisuustaloustiede  -  teollisuustaloustiede toimialatutkimus
661 needs to be added manually:  toimialatutkimus  -  toimialatutkimus  -  teollisuustaloustiede toimialatutkimus
673 needs to be added manually:  tyhjiöteollisuus  -  tyhjiöteollisuus  -  tyhjiöteollisuus
674 needs to be added manually:  eläingenetiikka  -  eläingenetiikka  -  eläingenetiikka
680 needs to be added manually:  yhteistyöpolitiikka  -  yhteistyöpolitiikka  -  yhteistyöpolitiikka
l2 metsätaloustiede
685 needs to be added manually:  kaksoistalous  -  kaksoistalous  

1034 needs to be added manually:  ympäristörikkomus  -  ympäristörikkomus  -  ympäristörikkomus
1038 needs to be added manually:  kaukoanalyysi  -  kaukoanalyysi  -  kaukokartoitus kaukoanalyysi
l2 eläinvahinko
1042 needs to be added manually:  metsävaurio  -  metsävaurio  -  metsävaurio
1044 needs to be added manually:  teollisuuspäästö  -  teollisuuspäästö  -  teollisuuden jätevesi teollisuuspäästö
l2 kasviyhdyskunta
1050 needs to be added manually:  kuminjalostus  -  kuminjalostus  -  kuminjalostus
1056 needs to be added manually:  vedensaaste  -  vedensaaste  -  vedensaaste
l2 oikeusvertailu
1070 needs to be added manually:  dokumentointijärjestelmä  -  dokumentointijärjestelmä  -  dokumentointijärjestelmä
l2 yrityspolitiikka
l2 ympäristöetiikka
1088 needs to be added manually:  hätäoikeus  -  hätäoikeus  -  hätäoikeus pakkotilaoikeus poikkeuslaki
1088 needs to be added manually:  pakkotilaoikeus  -  pakkotilaoikeus  -  hätäoikeus pakkotilaoikeus poikkeuslaki
l2 rannikkokalastus
l2

1528 needs to be added manually:  työonnettomuus  -  työonnettomuus  -  työonnettomuus
l2 talonvaltaaja
l2 kartografinen
1545 needs to be added manually:  ympäristökemikaali  -  ympäristökemikaaleja  -  ympäristökemikaaleja koskeva lainsäädäntö
1547 needs to be added manually:  meriluonnonvara  -  meriluonnonvarojen  -  meriluonnonvarojen suojelu
1552 needs to be added manually:  estuaariekosysteemi  -  estuaariekosysteemi  -  estuaariekosysteemi
1557 needs to be added manually:  kaukokartoituskeskus  -  kaukokartoituskeskus  -  kaukokartoituskeskus
1567 needs to be added manually:  juomavesivarasto  -  juomavesivarasto  -  juomavesivarasto
1570 needs to be added manually:  ympäristövalmiussuunnitelma  -  ympäristövalmiussuunnitelma  -  ympäristövalmiussuunnitelma
1571 needs to be added manually:  meriluonnonsuojelualiue  -  meriluonnonsuojelualiue  -  meriluonnonsuojelualiue
1574 needs to be added manually:  teollisuuskasvi  -  teollisuuskasvi  -  teollisuuskasvi
1579 needs to be adde

1835 needs to be added manually:  saastumaton  -  saastumattoman  -  saastumattoman ilman alue
1837 needs to be added manually:  ympäristönäytepankki  -  ympäristönäytepankki  -  ympäristönäytepankki
1838 needs to be added manually:  valintajalostus  -  valintajalostus  -  eläinten valintajalostus
1840 needs to be added manually:  ympäristönsuojeluneuvonta  -  ympäristönsuojeluneuvonta  -  ympäristönsuojeluneuvonta
l2 kuivatusjärjestelmä
1856 needs to be added manually:  jätekaasu  -  jätekaasu  -  jätekaasu
1860 needs to be added manually:  muuntamistekniikka  -  muuntamistekniikka  -  jätteen muuntamistekniikka
1862 needs to be added manually:  populaatiostatistikka  -  populaatiostatistikka  -  luonnonvaraisen eliöstön populaatiostatistikka
1866 needs to be added manually:  saastumisalue  -  saastumisalue  -  mahdollinen saastumisalue
1867 needs to be added manually:  joensuualue  -  joensuualue  -  suojeltava joensuualue
1868 needs to be added manually:  viitetietojärjestelmä  -  v

...

3686 needs to be added manually:  valuuttasuhde  -  valuuttasuhteet  -  valuuttasuhteet rahasuhteet
3686 needs to be added manually:  rahasuhde  -  rahasuhteet  -  valuuttasuhteet rahasuhteet
3687 needs to be added manually:  pakkotoimenpide  -  pakkotoimenpiteet  -  pakkotoimenpiteet
l2 halogenoida
3694 needs to be added manually:  valintajalostus  -  valintajalostus  -  kasvien valintajalostus
l2 bensiiniasema
l2 muistilaite
l2 liikennemelu
3709 needs to be added manually:  sytotoksisuus  -  sytotoksisuus  -  sytotoksisuus
3712 needs to be added manually:  asumisjäte  -  asumisjäte  -  talousjätteet asumisjäte
3715 needs to be added manually:  vesistötiede  -  vesistötiede  -  vesistötiede limnologia
l2 linkoaminen
3724 needs to be added manually:  minimointimahdollisuus  -  minimointimahdollisuudet  -  jätteen minimointimahdollisuudet
3725 needs to be added manually:  saasteanalyysi  -  saasteanalyysi  -  saasteanalyysi
l2 avolouhinta
l2 ylikansoitus
l2 hiekkatasanko
3732 needs to b

4095 needs to be added manually:  liuonnut  -  liuonnut  -  liuonnut happi
l2 pölynpoisto
4103 needs to be added manually:  jäännösanalyysi  -  jäännösanalyysi  -  jäännösanalyysi
l2 tavaraluettelo
l2 parasiitti
l2 polysyklinen
l2 saniteettilaite
l2 etologia
l2 säteilysairaus
l2 ioninvaihto
!!!!! error - not equal length:  [] ['']
4122 needs to be added manually:  sopetuvat  -  sopetuvat  -  sopetuvat lajit
l2 hajoamaton
l2 rasvahakuinen
w1 DNA
l2 biotekninen
l2 saastuttamaton
l2 saastepäästö
4136 needs to be added manually:  tulvasuojeluallas  -  tulvasuojeluallas  -  tulvasuojeluallas
4138 needs to be added manually:  valkaisusavi  -  valkaisusavi  -  valkaisusavi valkaisumaa
4138 needs to be added manually:  valkaisumaa  -  valkaisumaa  -  valkaisusavi valkaisumaa
4139 needs to be added manually:  säilytystankki  -  säilytystankki  -  hiilivedyn säilytystankki
l2 yliäänipamaus
4141 needs to be added manually:  äänivallipamaus  -  äänivallipamaus  -  yliäänipamaus äänivallipamaus
l2 

l2 saastuttamaton
4493 needs to be added manually:  kierrätyssuhde  -  kierrätyssuhde  -  kierrätyssuhde
l2 alkeistumallinen
l2 kaukolaina
4503 needs to be added manually:  todistelumenettely  -  todistelumenettely  -  todistelumenettely todisteiden talteenotto
4506 needs to be added manually:  öljynporausgeologi  -  öljynporausgeologia  -  öljynporausgeologia
4508 needs to be added manually:  suodatuspuristuskakku  -  suodatuspuristuskakku  -  suodatuspuristuskakku
4509 needs to be added manually:  pölyimmissio  -  pölyimmissio  -  pölyimmissio
l2 destruktio
4513 needs to be added manually:  ilmavoimajohto  -  ilmavoimajohto  -  ilmavoimajohto
l2 furaani
4518 needs to be added manually:  alisyklinen  -  alisyklinen  -  alisyklinen hiilivety
w1 Amerikka
4523 needs to be added manually:  saastenormi  -  saastenormi  -  saastenormi
l2 avustusmääräraha
4529 needs to be added manually:  halkeamisreaktio  -  halkeamisreaktio  -  atomiytimen halkeamisreaktio
4530 needs to be added manually: 

4956 needs to be added manually:  ontogenia  -  ontogenia  -  yksilönkehitys ontogenia
l2 patikkapolku
l2 kyllästysaine
l2 järjellisyys
4972 needs to be added manually:  sisäeritysoppi  -  sisäeritysoppi  -  sisäeritysoppi
l2 suolasuo
4975 needs to be added manually:  suolakenttä  -  suolakenttä  -  suolajärvi suolasuo suolakenttä
4977 needs to be added manually:  impaktori  -  impaktori  -  impaktori
4980 needs to be added manually:  karsinogeenisyys  -  karsinogeenisyys  -  karsinogeenisyys
4985 needs to be added manually:  tulivuoritiede  -  tulivuoritiede  -  tulivuoritiede vulkanologia
l2 vulkanologi
l2 lasivilla
l2 syövytys
l2 geotekniikka
4998 needs to be added manually:  jäätiköntutkimus  -  jäätiköntutkimus  -  jäätiköntutkimus jäätikkötiede
l2 jäätikkötiede
5002 needs to be added manually:  karsinogeenisyyskoe  -  karsinogeenisyyskoe  -  karsinogeenisyyskoe
5003 needs to be added manually:  jätteenpolttouuni  -  jätteenpolttouuni  -  jätteenpolttouuni
5008 needs to be added m

l2 astunta
5289 needs to be added manually:  polyklooribentsofuraani  -  polyklooribentsofuraani  -  polyklooribentsofuraani
l2 norsueläin
l2 bioetiikka
w1 YK
l2 kehityskonferenssi
l2 bioreaktori
l2 bioturvallisuus
872


In [20]:
# for manual correcting of words
pprint(words_to_manual_add)

{'aerosolipullo': 'aerosolipullo spraypurkki',
 'agroekosysteemi': 'agroekosysteemi',
 'agronominen': 'agronominen lisäaine',
 'aikabudjetti': 'aikabudjetti',
 'aineenkoestus': 'aineenkoestus',
 'ainejäämä': 'torjunta ainejäämä',
 'ainemenekki': 'ainemenekki',
 'ainevalvonta': 'torjunta ainevalvonnan normi',
 'aktinidi': 'aktinidi',
 'alatili': 'alatili',
 'alisyklinen': 'alisyklinen yhdiste',
 'alkalimaa': 'emäksinen maa alkalimaa',
 'aluelainsäädäntö': 'aluelainsäädäntö',
 'alumiinisäiliö': 'alumiinisäiliö',
 'amalgaama': 'amalgaama',
 'ammoniakkiutuminen': 'ammonifikaatio ammoniakkiutuminen',
 'analyysiohjelma': 'analyysiohjelma',
 'anomusoikeus': 'anomusoikeus',
 'asbestisementti': 'asbestisementti',
 'assimilaatiokyky': 'jätteen assimilaatiokyky',
 'asukasjakauma': 'asukasjakauma',
 'asumisjäte': 'talousjätteet asumisjäte',
 'asuntorahoituspalvelu': 'omaisuus ja asuntorahoituspalvelut',
 'asutusohjelma': 'asutusohjelma',
 'asutustarve': 'asutustarve',
 'atomiräjähdys': 'atomiräjäh

 'kauppajärjestely': 'kuplapolitiikka päästöjä koskeva kauppajärjestely',
 'kaupunkimelu': 'kaupunkimelu',
 'kaupunkitoimintaohjelma': 'kaupunkitoimintaohjelma',
 'kehityskaava': 'kehityskaava',
 'kehityssuunnittelu': 'kehityssuunnittelu',
 'keijusto': 'keijusto plankton',
 'keinoaine': 'keinoaine tekoaine',
 'keinoaineteollisuus': 'keinoaineteollisuus',
 'keinokasteluviljely': 'keinokasteluviljely',
 'keittolaite': 'kuumennin lämmitin keittolaite',
 'kemisorptio': 'kemisorptio kemiallinen adsorptio',
 'kertamenetelmä': 'kertamenetelmä eräprosessi',
 'kesantomaa': 'kesantomaa',
 'keskuspuistoalue': 'keskuspuistoalue',
 'kestohyödyke': 'kestokulutushyödyke kestohyödyke',
 'kidetiede': 'kidetiede',
 'kierrätyssuhde': 'kierrätyssuhde',
 'kiertoposti': 'kiertoposti',
 'kiertouuni': 'kiertouuni viertouuni',
 'kilpailukoe': 'kilpailukoe',
 'kirjaamisvelvollisuus': 'kirjaamisvelvollisuus',
 'kirjallisuustietopankki': 'kirjallisuustietopankki',
 'kirjanpitomenetelmä': 'kirjanpitomenetelmä',
 '

In [38]:
# the lemmas not in model were manually checked if they exist in other models in some variation
word = "fytomassa"
print(word)
try:
    model[word]
    print('model')
    pprint(model.most_similar(word))
except:
    pass

if word in big_lemmas_set:
    print('model_lemma_big', big_lemma_word_dic[word])

try:
    model_big[word]
    print('model big')
    pprint(model_big.most_similar(word))
except:
    pass

# if the original word with uppercase
if word in words_lower_orig:
    print('model big LOWER', words_lower_orig[word])
    pprint(model_big.most_similar(words_lower_orig[word]))
    

fytomassa


In [22]:
# for cases the lemmas aren't correct or don't exist in the dic
# manually made dictionary of "lemma" : "word which exists in some model" or "word1;word2" for the average
# e.g. 'mikroekosystém': 'mikro;ekosystém', 'bahný': 'bahno'
new_words_manually_checked = {}
    
len(new_words_manually_checked)

0

In [23]:
# function which finds the word in all the available models and returns it's vector (average vector)
def get_vector(word):
    # word is in small model
    try:
        return model[word]
    except:
        pass
    
    # word is in lemmas from big model - return average if the lemma was result of more words
    if word in big_lemmas_set:
        lemma_words_list = big_lemma_word_dic[word]
        tmp_vectors = []
        for word2 in lemma_words_list:
            tmp_vectors.append(model_big[word2])  
        return np.average(tmp_vectors, axis=0)

    try:
        return model_big[word]
    except:
        pass
    
    # if the original word with uppercase
    if word in words_lower_orig:
        orig_words_list = words_lower_orig[word]
        tmp_vectors = []
        for word2 in orig_words_list:
            tmp_vectors.append(model_big[word2])  
        return np.average(tmp_vectors, axis=0)

        #return model_big[words_lower_orig[word]]
    
    return []

In [24]:
# check if the word vectors are the same
#get_vector('Antarktyki') == get_vector('antarktyki')
get_vector('διβενζοφουράνιο')

[]

In [25]:
# ading words from manually checked list
new_vectors = []
new_words = []
for i, word in enumerate(new_words_manually_checked.keys()):
    
    tmp_vectors = []
    print(i, word)
    for tmp_word in new_words_manually_checked[word].split(';'):
        vector = get_vector(tmp_word)
        if vector != []:
            tmp_vectors.append(vector)
        else:
            print("XXX - some ERROR, word doesn't have vector, cannot add: ", tmp_word)
    if word not in new_words:
        new_vectors.append(np.average(tmp_vectors, axis=0))
        new_words.append(word)
print(len(new_vectors),len(new_words))        

0 0


In [26]:
# add new words to the model
if len(new_words) > 0:
    model.add(new_words,new_vectors)

In [27]:
print(len(topics),len(topics_nolemma))  

5305 5305


In [28]:
# 2nd run - checking if there are still unavailable words in topics
excluded_topics = []
#try if the topic-lemma words exist in the model   

for i, (topic, topic_nolemma) in enumerate(zip(topics, topics_nolemma)):
    # process the nolemma topic so we can see which word was lemmatized to which lemma
    topic_nolemma_orig = topic_nolemma.strip()
    topic_nolemma = re.sub('\W',' ',topic_nolemma)
    topic_nolemma = re.sub('\ +',' ',topic_nolemma)
    doc = nlp(topic_nolemma.strip().lower())
    nolemmas = []
    for token in doc:
        if token.is_stop:
            continue
        else:
            nolemmas.append(token.text)  
    
    topic_words = topic.split('_')
    
    if len(topic_words) != len(nolemmas):
        print("!!!!! error - not equal length: ", nolemmas, topic_words)
        continue
        
    found = 1
    for word, word_nolemma in zip(topic_words, nolemmas):
    #for word in topic_words:
        #print(word)
       
        vector = get_vector(word)
        if vector == []:
            print('nonexistend word: ', word, " - ", topic_nolemma)
            vector2 = get_vector(word_nolemma)
            if vector2 == []:
                found = 0
                print('nonexistend word: ', word_nolemma, " - ", topic_nolemma)
            else:
                print("Exist nolemma")

    if found == 0:
        print('!!! topic needs to be excluded: ', topic_nolemma_orig)
        if topic_nolemma.strip() not in excluded_topics:
            excluded_topics.append(topic_nolemma_orig)
                    
print(len(excluded_topics)) 

nonexistend word:  maisemansuunnittelu  -  maisemansuunnittelu 
nonexistend word:  maisemansuunnittelu  -  maisemansuunnittelu 
!!! topic needs to be excluded:  maisemansuunnittelu
nonexistend word:  resurssijohtaminen  -  resurssijohtaminen 
nonexistend word:  resurssijohtaminen  -  resurssijohtaminen 
!!! topic needs to be excluded:  resurssijohtaminen
nonexistend word:  maaeroosio  -  maaeroosio 
nonexistend word:  maaeroosio  -  maaeroosio 
!!! topic needs to be excluded:  maaeroosio


/home/dzon/kajo/kajo_semexp_core/lib/python3.6/site-packages/ipykernel_launcher.py:30: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


nonexistend word:  moottoriajoneuvoteollisuus  -  moottoriajoneuvoteollisuus 
nonexistend word:  moottoriajoneuvoteollisuus  -  moottoriajoneuvoteollisuus 
!!! topic needs to be excluded:  moottoriajoneuvoteollisuus
nonexistend word:  maaseutusuunnittelu  -  maaseutusuunnittelu 
nonexistend word:  maaseutusuunnittelu  -  maaseutusuunnittelu 
!!! topic needs to be excluded:  maaseutusuunnittelu
nonexistend word:  ruuanjalostusteollisuus  -  ruuanjalostusteollisuus 
nonexistend word:  ruuanjalostusteollisuus  -  ruuanjalostusteollisuus 
!!! topic needs to be excluded:  ruuanjalostusteollisuus
nonexistend word:  luonnonsuojelupolitiikka  -  luonnonsuojelupolitiikka 
nonexistend word:  luonnonsuojelupolitiikka  -  luonnonsuojelupolitiikka 
!!! topic needs to be excluded:  luonnonsuojelupolitiikka
nonexistend word:  tuotantokeino  -  maatalouden tuotantokeinot 
nonexistend word:  tuotantokeinot  -  maatalouden tuotantokeinot 
!!! topic needs to be excluded:  maatalouden tuotantokeinot
nonex

nonexistend word:  maanviljelystekniikka  -  maanviljelystekniikka 
nonexistend word:  maanviljelystekniikka  -  maanviljelystekniikka 
!!! topic needs to be excluded:  maanviljelystekniikka
nonexistend word:  kulttuuritavara  -  kulttuuritavarat 
nonexistend word:  kulttuuritavarat  -  kulttuuritavarat 
!!! topic needs to be excluded:  kulttuuritavarat
nonexistend word:  merigeologi  -  merigeologia 
nonexistend word:  merigeologia  -  merigeologia 
!!! topic needs to be excluded:  merigeologia
nonexistend word:  energialainsäädäntö  -  energialainsäädäntö 
nonexistend word:  energialainsäädäntö  -  energialainsäädäntö 
!!! topic needs to be excluded:  energialainsäädäntö
nonexistend word:  eläinperintö  -  eläinperintö 
nonexistend word:  eläinperintö  -  eläinperintö 
!!! topic needs to be excluded:  eläinperintö
nonexistend word:  ilmastovara  -  ilmastovarat 
nonexistend word:  ilmastovarat  -  ilmastovarat 
!!! topic needs to be excluded:  ilmastovarat
nonexistend word:  taustata

nonexistend word:  säädösperusteinen  -  säädösperusteinen valvonta 
nonexistend word:  säädösperusteinen  -  säädösperusteinen valvonta 
!!! topic needs to be excluded:  säädösperusteinen valvonta
nonexistend word:  ympäristöindeksi  -  ympäristöindeksi 
nonexistend word:  ympäristöindeksi  -  ympäristöindeksi 
!!! topic needs to be excluded:  ympäristöindeksi
nonexistend word:  liikepolitiikka  -  liikepolitiikka 
nonexistend word:  liikepolitiikka  -  liikepolitiikka 
!!! topic needs to be excluded:  liikepolitiikka
nonexistend word:  suojelupolitiikka  -  suojelupolitiikka 
nonexistend word:  suojelupolitiikka  -  suojelupolitiikka 
!!! topic needs to be excluded:  suojelupolitiikka
nonexistend word:  eläinvara  -  eläinvarat 
nonexistend word:  eläinvarat  -  eläinvarat 
!!! topic needs to be excluded:  eläinvarat
nonexistend word:  maakirjarekisteri  -  maakirjarekisteri lainhuutorekisteri 
nonexistend word:  maakirjarekisteri  -  maakirjarekisteri lainhuutorekisteri 
nonexistend

...

nonexistend word:  metsänviljelytekniikka  -  metsänviljelytekniikka 
nonexistend word:  metsänviljelytekniikka  -  metsänviljelytekniikka 
!!! topic needs to be excluded:  metsänviljelytekniikka
nonexistend word:  impulssikuormitus  -  impulssikuormitus 
nonexistend word:  impulssikuormitus  -  impulssikuormitus 
!!! topic needs to be excluded:  impulssikuormitus
nonexistend word:  avaruusilmailu  -  avaruusilmailu 
nonexistend word:  avaruusilmailu  -  avaruusilmailu 
!!! topic needs to be excluded:  avaruusilmailu
nonexistend word:  laivajäte  -  laivajäte 
nonexistend word:  laivajäte  -  laivajäte 
!!! topic needs to be excluded:  laivajäte
nonexistend word:  puolijuosta  -  puolijuokseva lanta 
nonexistend word:  puolijuokseva  -  puolijuokseva lanta 
!!! topic needs to be excluded:  puolijuokseva lanta
nonexistend word:  jäähdytysöljy  -  jäähdytysöljy 
nonexistend word:  jäähdytysöljy  -  jäähdytysöljy 
!!! topic needs to be excluded:  jäähdytysöljy
nonexistend word:  valatmeri

nonexistend word:  paleoekologia  -  paleoekologia 
nonexistend word:  paleoekologia  -  paleoekologia 
!!! topic needs to be excluded:  paleoekologia
nonexistend word:  assimilaatiokyky  -  jätteen assimilaatiokyky 
nonexistend word:  assimilaatiokyky  -  jätteen assimilaatiokyky 
!!! topic needs to be excluded:  jätteen assimilaatiokyky
nonexistend word:  saasteindikaattori  -  saasteindikaattori 
nonexistend word:  saasteindikaattori  -  saasteindikaattori 
!!! topic needs to be excluded:  saasteindikaattori
nonexistend word:  mädätyskammio  -  mädätyskammio sakokaivo 
nonexistend word:  mädätyskammio  -  mädätyskammio sakokaivo 
!!! topic needs to be excluded:  mädätyskammio, sakokaivo
nonexistend word:  sintrautuminen  -  sintraus sintrautuminen 
nonexistend word:  sintrautuminen  -  sintraus sintrautuminen 
!!! topic needs to be excluded:  sintraus, sintrautuminen
nonexistend word:  puhdistusjäännös  -  puhdistusjäännös 
nonexistend word:  puhdistusjäännös  -  puhdistusjäännös 
!

nonexistend word:  atomiräjähdys  -  atomiräjähdys 
nonexistend word:  atomiräjähdys  -  atomiräjähdys 
!!! topic needs to be excluded:  atomiräjähdys
nonexistend word:  välikannellinen  -  välikannellinen tankkeri 
nonexistend word:  välikannellinen  -  välikannellinen tankkeri 
!!! topic needs to be excluded:  välikannellinen tankkeri
nonexistend word:  merkitsemispakko  -  merkitsemispakko 
nonexistend word:  merkitsemispakko  -  merkitsemispakko 
!!! topic needs to be excluded:  merkitsemispakko
nonexistend word:  ylivuotokanava  -  ylivuotokanava 
nonexistend word:  ylivuotokanava  -  ylivuotokanava 
!!! topic needs to be excluded:  ylivuotokanava
nonexistend word:  jätteidenkuljetusauto  -  jätteidenkuljetusauto 
nonexistend word:  jätteidenkuljetusauto  -  jätteidenkuljetusauto 
!!! topic needs to be excluded:  jätteidenkuljetusauto
nonexistend word:  riskialtistus  -  riskialtistus 
nonexistend word:  riskialtistus  -  riskialtistus 
!!! topic needs to be excluded:  riskialtist

nonexistend word:  hiutaloittamisaine  -  hiutaloittamisaine 
nonexistend word:  hiutaloittamisaine  -  hiutaloittamisaine 
!!! topic needs to be excluded:  hiutaloittamisaine
nonexistend word:  terfenyyli  -  halogenoitu terfenyyli 
nonexistend word:  terfenyyli  -  halogenoitu terfenyyli 
!!! topic needs to be excluded:  halogenoitu terfenyyli
nonexistend word:  amalgaama  -  amalgaama 
nonexistend word:  amalgaama  -  amalgaama 
!!! topic needs to be excluded:  amalgaama
nonexistend word:  saprobisuusindeksi  -  saprobisuusindeksi 
nonexistend word:  saprobisuusindeksi  -  saprobisuusindeksi 
!!! topic needs to be excluded:  saprobisuusindeksi
nonexistend word:  mutageenisyystestaus  -  mutageenisyystestaus 
nonexistend word:  mutageenisyystestaus  -  mutageenisyystestaus 
!!! topic needs to be excluded:  mutageenisyystestaus
nonexistend word:  jauhamisjäännös  -  jauhamisjäännökset 
nonexistend word:  jauhamisjäännökset  -  jauhamisjäännökset 
!!! topic needs to be excluded:  jauha

nonexistend word:  reflektometria  -  reflektometria 
nonexistend word:  reflektometria  -  reflektometria 
!!! topic needs to be excluded:  reflektometria
nonexistend word:  itiökantainen  -  itiökantaiset 
nonexistend word:  itiökantaiset  -  itiökantaiset 
!!! topic needs to be excluded:  itiökantaiset
nonexistend word:  kiviöljy  -  kiviöljy raakaöljy 
nonexistend word:  kiviöljy  -  kiviöljy raakaöljy 
!!! topic needs to be excluded:  kiviöljy, raakaöljy
nonexistend word:  pehmennysaine  -  pehmennysaine plastisoimisaine 
nonexistend word:  pehmennysaine  -  pehmennysaine plastisoimisaine 
nonexistend word:  plastisoimisaine  -  pehmennysaine plastisoimisaine 
nonexistend word:  plastisoimisaine  -  pehmennysaine plastisoimisaine 
!!! topic needs to be excluded:  pehmennysaine, plastisoimisaine
nonexistend word:  virkavaltaistuminen  -  virkavaltaistuminen 
nonexistend word:  virkavaltaistuminen  -  virkavaltaistuminen 
!!! topic needs to be excluded:  virkavaltaistuminen
nonexist

In [29]:
# save topics containing words which are not present neither model - to be excluded from the topic list
print('excluded topics: ', len(excluded_topics))
pprint(excluded_topics)
with open(excluded_filename, 'wt', encoding='utf8') as f:
    for topic in excluded_topics:
        f.write(topic + "\n")

excluded topics:  862
['maisemansuunnittelu',
 'resurssijohtaminen',
 'maaeroosio',
 'moottoriajoneuvoteollisuus',
 'maaseutusuunnittelu',
 'ruuanjalostusteollisuus',
 'luonnonsuojelupolitiikka',
 'maatalouden tuotantokeinot',
 'ympäristönsuojelusääntö',
 'kesantomaa',
 'maankäyttöjärjestelmä',
 'rannikkoympäristö',
 'maaperän suolautuminene',
 'vesiekologia',
 'meriekologia',
 'maisemanhoito, maisemansuunnittelu',
 'maisemaekologia',
 'maatalousbiotekniikka',
 'maaympäristö',
 'luottoapu',
 'metsätuotanto',
 'vihannesten vilejly',
 'maatalouslainsäädäntö',
 'maatalousvarasto',
 'maataloustiede, maanviljelystaloustiede',
 'ravintotekniikka',
 'maatalousekologia',
 'kehityssuunnittelu',
 'maatalousvälineistö',
 'teollisuuslainsäädäntö',
 'rantaeroosio',
 'lannantuotanto',
 'maaperäkivennäistiede',
 'tihkuvesi',
 'maaseutuarkkitehtuuri',
 'valtakunnansuunnittelu',
 'maannostuminen',
 'yksilöliikenne',
 'meren pohjakerrostuma, merisedimentti',
 'rannikkomeri, rantavesi',
 'vesien suojjelu

In [30]:
# create new lists which don't containt topics with non-existent words

# load list with excluded topics
with open(excluded_filename, 'rt', encoding='utf8') as fp:
    tmp_topics = fp.readlines()
excluded_set = set()
for topic in tmp_topics:
    excluded_set.add(topic.strip())

with open(nolemma_list_file, 'rt', encoding='utf8') as fp:
        topics_file = fp.readlines()

topics_final = []
topics_lemma = []

excluded = 0
for topic in topics_file:
    topic_orig = topic.strip()
    if topic_orig in excluded_set:
        print("=== EXCLUDE ", topic_orig)
        excluded += 1
        continue
    topic = re.sub('\W',' ',topic)
    topic = re.sub('\ +',' ',topic)
    doc = nlp(topic.strip().lower())
    lemmas = []

    for token in doc:
        if token.is_stop:
            continue
        lemmas.append(token.lemma_)    
    topics_lemma.append("_".join(lemmas))
    topics_final.append(topic_orig)

    print(topic_orig, ' --- ', "_".join(lemmas))

print('Excluded: ', excluded)
print('Final topics: ', len(topics_lemma), len(topics_final))
    
with open(lemma_list_file_excluded, 'wt', encoding='utf8') as f:
    for topic_lemma in topics_lemma:
        f.write(topic_lemma + "\n")
with open(nolemma_list_file_excluded, 'wt', encoding='utf8') as f:
    for topic_final in topics_final:
        f.write(topic_final + "\n")  

paikallinen kehitys  ---  paikallinen_kehitys
maaseuturakentaminen  ---  maaseuturakentaminen
vesitalous  ---  vesitalous
koulutustaso  ---  koulutustaso
yleissivistävä koulutus  ---  yleissivistää_koulutus
rantapenger  ---  rantapenger
yhteisöoikeus  ---  yhteisöoikeus
kulttuuriperintö  ---  kulttuuriperintö
viljelysmaa, maatalousmaa  ---  viljelysmaa_maatalousmaa
maatalousmaisema  ---  maatalousmaisema
maataloustuotanto  ---  maataloustuotanto
taloudellinen kehitys  ---  taloudellinen_kehitys
ravintoketju  ---  ravintoketju
elintarviketeollisuus  ---  elintarviketeollisuus
ruuan laatu  ---  ruoka_laatu
globaali lämpeneminen  ---  globaali_lämpeneminen
kasvihuoneilmiö  ---  kasvihuoneilmiö
elinkeinoelämän kehitys  ---  elinkeinoelämä_kehitys
työmarkkinat  ---  työmarkkina
pääsy maa-alueelle  ---  pääsy_maa_alue
uusjako, tilusten sijoitus  ---  uusjako_tilus_sijoitus
maankäytön suunnittelu  ---  maankäyttö_suunnittelu
maisema-arkkitehtuuri  ---  maisema_arkkitehtuuri
=== EXCLUDE  maise

yhteisön säädös  ---  yhteisö_säädös
maatalousrakennus  ---  maatalousrakennus
teollisuusrakennus  ---  teollisuusrakennus
ilmasto  ---  ilmasto
kosmeettinen teollisuus  ---  kosmeettinen_teollisuus
karjanhoito  ---  karjanhoito
voimavarojen käyttö  ---  voimavara_käyttö
veden käyttö  ---  vesi_käyttö
nurmi  ---  nurmi
terästeollisuus  ---  terästeollisuus
sähköteollisuus  ---  sähköteollisuus
vesistöalueen suunnitelmallinen käyttö  ---  vesistöalue_suunnitelmallinen_käyttö
ekosysteemin rappeutuminen  ---  ekosysteemi_rappeutuminen
vaatetusteollisuus  ---  vaatetusteollisuus
keramiikkateollisuus  ---  keramiikkateollisuus
markkinatalous  ---  markkinatalous
ympäristön hoito  ---  ympäristö_hoito
ohjelmiston kehitys  ---  ohjelmisto_kehitys
=== EXCLUDE  maatalousvarasto
maanviljelys  ---  maanviljelys
maataloustuote  ---  maataloustuote
ilmaston muutos  ---  ilmasto_muutos
=== EXCLUDE  maataloustiede, maanviljelystaloustiede
maankäyttö  ---  maankäyttö
teollinen suunnittelu  ---  teolli

maisemansuojelu  ---  maisemansuojelu
maisemansuojelu  ---  maisemansuojelu
merikuljetus  ---  merikuljetus
kaivosteollisuus  ---  kaivosteollisuus
eläinfysiologia  ---  eläinfysiologi
käsiteollisuus  ---  käsiteollisuus
maavero, kiinteistövero  ---  maavero_kiinteistövero
ilmatilan suunnittelu  ---  ilmatila_suunnittelu
maa-alue  ---  maa_alue
meluvaikutus  ---  meluvaikutus
=== EXCLUDE  maanviljelystekniikka
väestönkasvu  ---  väestönkasvu
viljavuus  ---  viljavuus
tiedepolitiikka  ---  tiedepolitiikka
maanviljelys ilman kastelulaitteita  ---  maanviljelys_kastelulaite
kannanvaihtelu  ---  kannanvaihtelu
EY:n direktiivi  ---  ey_n_direktiivi
=== EXCLUDE  kulttuuritavarat
ympäristön kemikaalit  ---  ympäristö_kemikaali
oikeudellinen sääntely  ---  oikeudellinen_sääntely
alueellinen sääntely  ---  alueellinen_sääntely
ympäristöpolitiikka  ---  ympäristöpolitiikka
luonnonmetsä  ---  luonnonmetsä
nahkateollisuus  ---  nahkateollisuus
puhdistus maan läpi  ---  puhdistus_maa
maan parannus,

maakerros  ---  maakerros
=== EXCLUDE  teollisuustaloustiede, toimialatutkimus
maan ekologia  ---  maa_ekologia
sosiaaliset puitteet  ---  sosiaalinen_puite
vesieläin  ---  vesieläin
biologiset resurssit  ---  biologinen_resurssi
elinympäristö  ---  elinympäristö
sosiaaliset menot  ---  sosiaalinen_meno
markkinat  ---  markkina
maan arvo  ---  maa_arvo
työllisyys  ---  työllisyys
yksityinen hallinta-alue  ---  yksityinen_hallinta_alue
valtion maa  ---  valtio_maa
=== EXCLUDE  tyhjiöteollisuus
=== EXCLUDE  eläingenetiikka
rakennustekniikka  ---  rakennustekniikka
koulutuksen suunnittelu  ---  koulutus_suunnittelu
uusi kaupunki  ---  kaupunki
maantieteellinen jakauma  ---  maantieteellinen_jakauma
kulttuuri, sivistys  ---  kulttuuri_sivistys
=== EXCLUDE  yhteistyöpolitiikka
metsätalous  ---  metsätalous
metsätaloustiede  ---  metsätaloustiede
maaperän saaste  ---  maaperä_saaste
kunnossapito  ---  kunnossapito
=== EXCLUDE  kaksoistalous
kansanperinne  ---  kansanperinne
väestö  ---  väes

sisäympäristö  ---  sisäympäristö
työoikeus  ---  työoikeus
=== EXCLUDE  ympäristöterminologia
maaseutuasutus  ---  maaseutuasutus
arktinen alue  ---  arktinen_alue
historiallinen kehitys  ---  historiallinen_kehitys
ympäristöinvestointi  ---  ympäristöinvestointi
=== EXCLUDE  ensisijaistaminen ympäristön näkökulmasta
ympäristöä koskevat tiedot  ---  ympäristö_koskea_tieto
luonnononnettomuus  ---  luonnononnettomuus
=== EXCLUDE  ympäristökannustin
asutus(taajama)  ---  asutus_taajama
=== EXCLUDE  kaavoitustoimenpide, suunnittelutoimenpide
eläinten käyttäytyminen  ---  eläin_käyttäytyminen
markkinatutkimus  ---  markkinatutkimus
kauppapuutarhan pito  ---  kauppapuutarha_pito
kasvisto  ---  kasvisto
=== EXCLUDE  täyssuolanpoisto
lain täytäntöönpano  ---  laki_täytäntöönpano
nuorisotyö  ---  nuorisotyö
voimaanpanolaki  ---  voimaanpanolaki
huoli ympäristöstä  ---  huoli_ympäristö
julkinen laitos  ---  julkinen_laitos
hiiliteollisuus  ---  hiiliteollisuus
=== EXCLUDE  teollisuusliete
yhtei

...

osakepääoma  ---  osakepääoma
tiheys  ---  tiheys
desinfiointi  ---  desinfiointi
maiseman tuhoutuminen  ---  maisema_tuhoutuminen
tietopaketti  ---  tietopaketti
polysyklinen aromaattinen hiilivety  ---  polysyklinen_aromaattinen_hiilivety
kivihiilen louhiminen  ---  kivihiili_louhiminen
käytöstä poistettu sotilasalue  ---  käyttö_poistaa_sotilasalue
ristiriita  ---  ristiriita
jäännösten kierrätys  ---  jäännös_kierrätys
vetysulfidi, rikkivety  ---  vetysulfidi_rikkivety
sorsalinnut  ---  sorsalintu
=== EXCLUDE  tullitariffi, tullitaksa
=== EXCLUDE  vaahdotusaine
=== EXCLUDE  kasvisosiologia
=== EXCLUDE  kartanpiirustus
saasteen poisto  ---  saaste_poisto
tavoitteiden asettaminen  ---  tavoite_asettaminen
vaarasta ilmoittaminen  ---  vaara_ilmoittaminen
rasteri  ---  rasteri
kansalaisoikeudet  ---  kansalaisoikeus
aids  ---  AIDS
=== EXCLUDE  paleoekologia
tupakka  ---  tupakka
hätämajoitus  ---  hätämajoitus
=== EXCLUDE  jätteen assimilaatiokyky
=== EXCLUDE  saasteindikaattori
=== E

vulkanismi, vulkaaniset ilmiöt  ---  vulkanismi_vulkaaninen_ilmiö
valvomaton kaatopaikka  ---  valvomaton_kaatopaikka
inertiksi muunto  ---  inertti_muunto
laimennettu happo, laimea happo  ---  laimeta_happo_laimea_happo
kuulemismenettely  ---  kuulemismenettely
suurin sallittu pitoisuus  ---  sallittu_pitoisuus
puro  ---  puro
laitos, johon tarvitaan hyväksyntä  ---  laitos_tarvita_hyväksyntä
ydinvoimala  ---  ydinvoimala
tuotevertailu  ---  tuotevertailu
saastuttajien toteaminen  ---  saastuttaja_toteaminen
tahnamainen jäte  ---  tahnamainen_jäte
tiedonlähde  ---  tiedonlähde
myrkky, toksiini  ---  myrkky_toksiini
mylly  ---  mylly
=== EXCLUDE  monispektriskanneri
imeytyminen  ---  imeytyminen
=== EXCLUDE  öljynjäännösten talteenotto
hallitseva  ---  hallitseva
hälytys  ---  hälytys
=== EXCLUDE  puukuorman purkaminen
laskenta  ---  laskenta
päätöslauselma  ---  päätöslauselma
polttomoottori  ---  polttomoottori
epämetallit  ---  epämetalli
laboratoriokoe  ---  laboratoriokoe
puutarha

lisäaineen päivittäinen saanti ilman terveydellistä vaaraa  ---  lisäaine_päivittäinen_saanti_terveydellinen_vaara
ilmoitus, tiedotus  ---  ilmoitus_tiedotus
liikennevälineet, kulkuyhteys  ---  liikenneväline_kulkuyhteys
värjätä, väriaine  ---  värjätä_väriaine
vaikeasti käsiteltävä jäte  ---  vaikeasti_käsitellä_jäte
=== EXCLUDE  märkäerotin
=== EXCLUDE  dekantointi, kuortaminen
=== EXCLUDE  huvilaivamatkailu
=== EXCLUDE  eliöntorjunta-aine
=== EXCLUDE  valutusjärjestelmä
liukenematon aine  ---  liukenematon_aine
säteily  ---  säteily
ministeriö  ---  ministeriö
veden virtausta koskeva tutkimus ja mittaus  ---  vesi_virtaus_koskea_tutkimus_mittaus
Oseania  ---  Oseania
ympäristöön rikastuva tekijä  ---  ympäristö_rikastaa_tekijä
sudenkorennot  ---  sudenkorento
=== EXCLUDE  tietoväline, tiedonsiirtoväline
suljettu alue  ---  sulkea_alue
hieno  ---  hieno
riita-asia, käräjöinti  ---  riita_käräjöinti
hajanainen lähde  ---  hajanainen_lähde
tuhka  ---  tuhka
kromi  ---  kromi
tina  --- 

suisto  ---  suisto
annos, annostus  ---  annos_annostus
pentakloorifenoli  ---  pentakloorifenoli
lajittelu syntypaikalla (jäte)  ---  lajittelu_syntypaikka_jäte
koulutyö  ---  koulutyö
poisto  ---  poisto
jokisokeus  ---  jokisokeus
emulsio  ---  emulsio
savupiippujen liiallinen korkeus  ---  savupiippu_liiallinen_korkeus
=== EXCLUDE  yksilönkehitys, ontogenia
osoitin  ---  osoittaa
kuulon tarkkuus  ---  kuulo_tarkkuus
patikkapolku  ---  patikkapolku
itu, itiö, alkio, mikrobi  ---  itu_itiö_alkio_mikrobi
torakat  ---  torakka
jätteiden kaato  ---  jäte_kaato
pato, sulku  ---  pato_sulku
kyllästysaine  ---  kyllästysaine
hopea  ---  hopea
röntgensäde  ---  röntgensäde
saniaiset  ---  saniainen
Ames-testi  ---  ames_testi
savu, höyry, savustaa, savuta  ---  savu_höyry_savustaa_savuta
lyijytön bensiini  ---  lyijytön_bensiini
järjellisyys  ---  järjellisyys
=== EXCLUDE  sisäeritysoppi
uraani  ---  ura
satama  ---  satama
=== EXCLUDE  suolajärvi, suolasuo, suolakenttä
räjähtävä  ---  räj

jäätikön massatase  ---  jäätikkö_massatase
maaperän huonontuminen  ---  maaperä_huonontuminen
kaatopaikkadirektiivi  ---  kaatopaikkadirektiivi
meriliikenteen polttoaine  ---  meriliikenne_polttoaine
=== EXCLUDE  meristrategiapuitedirektiivi
yhdyskuntajätteen synty  ---  yhdyskuntajäte_synty
yhdyskuntajätevesi  ---  yhdyskuntajätevesi
nitraattidirektiivi  ---  nitraattidirektiivi
eloperäinen maanparannusaine  ---  eloperäinen_maanparannusaine
organisaation ympäristöjalanjälkeä koskeva menetelmä  ---  organisaatio_ympäristöjalanjälki_koskea_menetelmä
otsonikato  ---  otsonikato
=== EXCLUDE  puistojäte
=== EXCLUDE  kasvibiostimulantti
poliittinen kehys  ---  poliittinen_kehys
politiikan täytäntöönpano  ---  politiikka_täytäntöönpano
primäärienergia  ---  primäärienergia
tuotteen ympäristöjalanjälkeä koskeva menetelmä  ---  tuote_ympäristöjalanjälki_koskea_menetelmä
käyttäjä-tuottajuus  ---  käyttäjä_tuottajuus
uusiutuvia energialähteitä koskeva direktiivi  ---  uusiutuva_energialähde_ko

In [31]:
# adding all the words and topics to model

# load the lemma list
with open(lemma_list_file_excluded) as fp:
    topics = fp.readlines()
# load the nolemma list
with open(nolemma_list_file_excluded) as fp:
    topics_nolemma = fp.readlines()    
    

#try if the topic-lemma words exist in the model     
new_vectors = []
new_words = []
for i, (topic, topic_nolemma) in enumerate(zip(topics, topics_nolemma)):
#for i,topic in enumerate(topics):
    topic_nolemma = re.sub('\W',' ',topic_nolemma)
    topic_nolemma = re.sub('\ +',' ',topic_nolemma)
    doc = nlp(topic_nolemma.strip().lower())
    nolemmas = []
    for token in doc:
        if token.is_stop:
            continue
        else:
            nolemmas.append(token.text) 
            
    topic = topic.strip()
    topic_words = topic.split('_')
    
    if len(topic_words) != len(nolemmas):
        print("!!!!! error - not equal length: ", nolemmas, topic_words)
        continue
    #if len(topic_words)<2:
    #    continue
    
    
    tmp_vectors = []
    
    for word, word_nolemma in zip(topic_words, nolemmas):
    #for word in topic_words:
        vector = get_vector(word)
        
        try:
            tmp_vectors.append(model[word])
        except: 
            print('\nNOT in vocabulary: ', word, topic)
            vector = get_vector(word)
            if vector != []:
                tmp_vectors.append(vector)
                if word not in new_words:
                    print('append ', word)
                    new_vectors.append(vector)
                    new_words.append(word)   
            else:
                vector = get_vector(word_nolemma)
                if vector != []:
                    tmp_vectors.append(vector)
                    if word not in new_words:
                        print('append ', word)
                        new_vectors.append(vector)
                        new_words.append(word)  
                else:
                    raise Warning(f"XXX {word} {topic} {topic_nolemma}")
                    print("XXX", word, topic, topic_nolemma) 
    #if word not in new_words:
    #    new_vectors.append(list(np.average(tmp_vectors, axis=0)))
    #    new_words.append(word)        
    if len(tmp_vectors) > 1:
        if topic not in new_words:
   
            print('append ', topic)
            new_vectors.append(list(np.average(tmp_vectors, axis=0)))
            new_words.append(topic)
        else:
            print('doubled TOPIC: ', topic)    
    else:
        print('--- only one word, not a new topic>', topic)

append  paikallinen_kehitys
--- only one word, not a new topic> maaseuturakentaminen
--- only one word, not a new topic> vesitalous
--- only one word, not a new topic> koulutustaso
append  yleissivistää_koulutus

NOT in vocabulary:  rantapenger rantapenger
append  rantapenger
--- only one word, not a new topic> rantapenger

NOT in vocabulary:  yhteisöoikeus yhteisöoikeus
append  yhteisöoikeus
--- only one word, not a new topic> yhteisöoikeus
--- only one word, not a new topic> kulttuuriperintö
append  viljelysmaa_maatalousmaa

NOT in vocabulary:  maatalousmaisema maatalousmaisema
append  maatalousmaisema
--- only one word, not a new topic> maatalousmaisema
--- only one word, not a new topic> maataloustuotanto
append  taloudellinen_kehitys
--- only one word, not a new topic> ravintoketju
--- only one word, not a new topic> elintarviketeollisuus
append  ruoka_laatu
append  globaali_lämpeneminen
--- only one word, not a new topic> kasvihuoneilmiö
append  elinkeinoelämä_kehitys
--- only on

/home/dzon/kajo/kajo_semexp_core/lib/python3.6/site-packages/ipykernel_launcher.py:47: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


append  vesivara_kehittäminen
append  teknillinen_viljelyskasvi
append  vesivara_suojelu
append  vesi_laatu_hallinta
append  pelto_viljely
--- only one word, not a new topic> maatalouspolitiikka
--- only one word, not a new topic> elintarviketuotanto
append  rahoitusinstrumentti_väline
--- only one word, not a new topic> kaupunkisuunnittelu
append  pintavesi_hallinta
--- only one word, not a new topic> vesivara
append  luonnonvara_hallinta
append  luonnonvarainen_alue_suojelu

NOT in vocabulary:  luonnonsuojelujärjestö luonnonsuojelujärjestö
append  luonnonsuojelujärjestö
--- only one word, not a new topic> luonnonsuojelujärjestö
--- only one word, not a new topic> kaupunkialue
append  maaseutu_haja_asutusalue
append  kunnallinen_vesihuolto
--- only one word, not a new topic> pintavesi
append  maankäyttö_luokitus
append  vesi_määrä_hallinta

NOT in vocabulary:  Eurooppa Eurooppa_unioni
append  Eurooppa
append  Eurooppa_unioni
append  julkinen_palvelu
--- only one word, not a new topic>

--- only one word, not a new topic> kulttuuripolitiikka
--- only one word, not a new topic> hotelliala
append  vesi_tarve
append  talous_alkaa
append  kriittinen_taso
append  metsä_ekosysteemi
--- only one word, not a new topic> rannikkoalue
--- only one word, not a new topic> maatalous
append  vesi_hinnoittelu

NOT in vocabulary:  maakuljetus maakuljetus
append  maakuljetus
--- only one word, not a new topic> maakuljetus
append  julkinen_liikenne
append  kuiva_maa
append  luonnonvara_hyväksikäyttö
append  vesi_laatu
--- only one word, not a new topic> aikuiskoulutus
append  vara_suojelu
append  vesi_laatu_ennallaan_palauttaminen
--- only one word, not a new topic> ympäristövaikutus
append  puu_poisto_maapaikka
append  kaupunki_saastuminen
append  kalatalous_kalastuselinkeino
append  jokiranta_kasvillisuus
--- only one word, not a new topic> ympäristötekniikka
append  liittohallitus_laki
append  taloudellinen_elinkelpoisuus

NOT in vocabulary:  sosiaalilääketiede sosiaalilääketiede
app

append  osallistuminen_yhteiskunta_toiminta
append  kauppaoikeus_talousoikeus
--- only one word, not a new topic> ravinnontarve
append  yksityinen_sektori
append  julkinen_sektori
append  julkinen_osallistuminen
append  julkinen_meno_valtio_meno
append  historia_tutkimus
--- only one word, not a new topic> ympäristöarviointi
--- only one word, not a new topic> elinkaari
--- only one word, not a new topic> metsäteollisuus
--- only one word, not a new topic> metsäpolitiikka
append  vaikutus_ympäristö
append  sosiaalinen_edustus
append  tekninen_ohje
append  orgaaninen_jäte
append  biologinen_tiede
append  tehokkuus_parantaminen
--- only one word, not a new topic> sosiaaliturva
--- only one word, not a new topic> terveydensuojelu
append  eläin_ruokinta
--- only one word, not a new topic> kalanviljely
--- only one word, not a new topic> ympäristökasvatus
--- only one word, not a new topic> vuorovesi

NOT in vocabulary:  eläinekologia eläinekologia
append  eläinekologia
--- only one word, n

...

--- only one word, not a new topic> liuotin
append  ajoittainen_melu
append  ydinonnettomuus_vaara

NOT in vocabulary:  tuotevastuu tuotevastuu
append  tuotevastuu
--- only one word, not a new topic> tuotevastuu
append  onnettomuus_aiheuttaja
--- only one word, not a new topic> katamaraani
--- only one word, not a new topic> aminohappo
--- only one word, not a new topic> ydinase
--- only one word, not a new topic> karbonaatti
--- only one word, not a new topic> kotisivu

NOT in vocabulary:  merikortti karsi_merikortti
append  merikortti
append  karsi_merikortti
append  bibliografinen_tieto
append  auer_sumentaa
append  toimia_aine
--- only one word, not a new topic> hyönteissyöjä

NOT in vocabulary:  kartografinen kartografinen_tietojärjestelmä_laboratorio
append  kartografinen_tietojärjestelmä_laboratorio
--- only one word, not a new topic> sinkki
append  tie_tielinja
--- only one word, not a new topic> mikrobiologi
append  mainos_ilmoitus
append  kaatopaikka_pohja_eristäminen

NOT in

--- only one word, not a new topic> tammi
append  aromaattinen_aine
--- only one word, not a new topic> ennuste

NOT in vocabulary:  osmoosi osmoosi
append  osmoosi
--- only one word, not a new topic> osmoosi
--- only one word, not a new topic> kartasto
append  dokumentointi_asiakirja_aineisto
--- only one word, not a new topic> jyrsijä
--- only one word, not a new topic> bakteriologia
append  elektroninen_tieto_vaihto
--- only one word, not a new topic> jalostamo
append  hyväksyminen_hyväksyntä
--- only one word, not a new topic> allergia
append  lakka_vernissa
append  toimeksianto_tehtävä
append  katu_puhtaanapito
--- only one word, not a new topic> kartoittaa
--- only one word, not a new topic> syanidi
--- only one word, not a new topic> lyijy
--- only one word, not a new topic> kipsi
--- only one word, not a new topic> puhdistus

NOT in vocabulary:  sulkuportti sulkuportti
append  sulkuportti
--- only one word, not a new topic> sulkuportti
append  ympäristötietoisuus_ympäristöliike

append  virkatehtävä_virkavelvollisuus
--- only one word, not a new topic> vandalismi
append  hermosto_myrkyllisyys
--- only one word, not a new topic> lakka
--- only one word, not a new topic> kumi
append  radioaktiivinen_alkuaine

NOT in vocabulary:  liuotus liuotus_liukeneminen
append  liuotus
append  liuotus_liukeneminen
append  mutageeninen_aine
--- only one word, not a new topic> vertailu

NOT in vocabulary:  sekoittuvuus sekoittuvuus
append  sekoittuvuus
--- only one word, not a new topic> sekoittuvuus
append  palautuminen_siirtyminen
--- only one word, not a new topic> putous
append  maankuori_hidas_liike
append  maanmittaus_geodesia
--- only one word, not a new topic> tiili
--- only one word, not a new topic> bensiinimoottori
append  mikro_organismi_mikrobi
--- only one word, not a new topic> hitsaus
append  hiljainen_työmatkaliikenne
--- only one word, not a new topic> onkiminen
--- only one word, not a new topic> heijastuma
--- only one word, not a new topic> piiru
--- only 

--- only one word, not a new topic> satama
--- only one word, not a new topic> räjähtävä
--- only one word, not a new topic> lasite
append  rikastaa_ura
--- only one word, not a new topic> termiitti
append  cfc_yhdiste
--- only one word, not a new topic> mutantti
--- only one word, not a new topic> tislaus
--- only one word, not a new topic> lamppu

NOT in vocabulary:  lasivilla lasivilla
append  lasivilla
--- only one word, not a new topic> lasivilla
--- only one word, not a new topic> beryllium

NOT in vocabulary:  syövytys etsaus_syövytys
append  syövytys
append  etsaus_syövytys
--- only one word, not a new topic> desibeli
--- only one word, not a new topic> uuni
--- only one word, not a new topic> amiini

NOT in vocabulary:  geotekniikka geotekniikka
append  geotekniikka
--- only one word, not a new topic> geotekniikka
--- only one word, not a new topic> feromoni
--- only one word, not a new topic> isotooppi
append  hävitä_laji
--- only one word, not a new topic> muste
--- only one

append  ekologinen_sivilisaatio
append  ekologinen_jalanjälki

NOT in vocabulary:  häiriönsietokyky ympäristö_häiriönsietokyky
append  häiriönsietokyky
append  ympäristö_häiriönsietokyky

NOT in vocabulary:  vaihettumisvyöhyke vaihettumisvyöhyke
append  vaihettumisvyöhyke
--- only one word, not a new topic> vaihettumisvyöhyke
append  ekosysteemi_vuorovaikutus
--- only one word, not a new topic> ekosysteemipalvelu
append  ekosysteemipalvelu_haavoittuvuus
append  tehokkuus_arviointi
--- only one word, not a new topic> päästöoikeus

NOT in vocabulary:  päästötaso päästötaso
--- only one word, not a new topic> päästötaso
append  päästökauppa_järjestelmä
append  ympäristöala_toimintaohjelma
--- only one word, not a new topic> ympäristöjalanjälki
--- only one word, not a new topic> ympäristöhallinto
--- only one word, not a new topic> ympäristölainsäädäntö

NOT in vocabulary:  EU EU_n_päästökauppajärjestelmä
append  EU_n_päästökauppajärjestelmä

NOT in vocabulary:  Eurooppa Eurooppa_ympärist

In [33]:
# add topics to model
model.add(new_words,new_vectors)

In [34]:
# save the model with new topics
model.save_word2vec_format(excluded_model_name, binary=True)
#reload the model  otherwise cannot find similarities fo newly added words
model = KeyedVectors.load_word2vec_format(excluded_model_name, binary=True)

In [35]:
len(model.index2word)

111253

In [39]:
# try the model
model.similar_by_vector('liiallinen_lannoittaminen')

[('liiallinen', 0.8417060971260071),
 ('liiallinen_hyväksikäyttö', 0.840916633605957),
 ('lannoittaminen', 0.8333126306533813),
 ('luonnollinen_lannoite', 0.7493617534637451),
 ('kemiallinen_lannoite', 0.7150038480758667),
 ('tuholainen_aiheuttaa_ruoka_pilaantuminen', 0.7139127850532532),
 ('kemiallinen_tuholainen_torjunta', 0.703312337398529),
 ('lisäaine_päivittäinen_saanti_terveydellinen_vaara', 0.6989914178848267),
 ('liika', 0.6983906626701355),
 ('maatalous_aiheuttaa_saastuminen', 0.6918289065361023)]

In [40]:
# find most similar to topics to terms in the new model

# load the new lemma list
with open(lemma_list_file_excluded) as fp:
    topics = fp.readlines()

for i, topic in enumerate(topics):
    topic = topic.strip()
    print('\n-----\n', topic)
    try:
        print(i, model.similar_by_vector(topic))
    except:
        print(topic, 'NOT in model')


-----
 paikallinen_kehitys
0 [('taloudellinen_kehitys', 0.884630560874939), ('sosiaalinen_kehitys', 0.8804570436477661), ('historiallinen_kehitys', 0.8774236440658569), ('kehitys', 0.8648649454116821), ('ympäristö_kehitys', 0.860937237739563), ('kulttuuri_kehitys', 0.8561224341392517), ('ala_viimeinen_kehitys_vallita_tekninen_kehitystaso', 0.8536185026168823), ('elinkeinoelämä_kehitys', 0.8521822690963745), ('ekologinen_kehitys', 0.8490339517593384), ('paikallinen_resurssi_hyödyntäminen', 0.8305875658988953)]

-----
 maaseuturakentaminen
1 [('maaseuturahoitus', 0.6721558570861816), ('hajarakentaminen', 0.6211841106414795), ('puurakentaminen', 0.6003641486167908), ('aluerakentaminen', 0.5936902761459351), ('tuulivoimarakentaminen', 0.5828033685684204), ('pientalorakentaminen', 0.5743743181228638), ('palvelurakentaminen', 0.5575456023216248), ('rakentaminen', 0.5520482063293457), ('kalliorakentaminen', 0.5473611354827881), ('kaupunkirakentaminen', 0.5452227592468262)]

-----
 vesitalous

35 [('keskiaste', 0.9225724935531616), ('koulutus', 0.7950879335403442), ('peruskoulutus', 0.7877575159072876), ('alkeisopetus_perusaste_koulutus', 0.7764261960983276), ('yleissivistää_koulutus', 0.7761905789375305), ('yliopistotaso_koulutus', 0.7541100978851318), ('ammattikoulutus', 0.7490333914756775), ('koulutusaste', 0.737169623374939), ('koulutusala', 0.7299893498420715), ('koulutustaso', 0.7191504240036011)]

-----
 maa_tiivistys
36 [('puhdistus_maa', 0.8800258636474609), ('maa', 0.8796316385269165), ('maa_kosteus', 0.8652142286300659), ('kuiva_maa', 0.8620829582214355), ('maa_puhdistaminen', 0.8617476224899292), ('maa_lujitus_maa_stabilointi', 0.8588136434555054), ('maa_alue_kunnostus', 0.8583489060401917), ('maa_alue', 0.8517445921897888), ('maa_kuormitus', 0.8477222919464111), ('maa_saneeraus', 0.8467265367507935)]

-----
 maa_kasvukunto_heikkeneminen
37 [('kasvukunto', 0.8205385804176331), ('luonnonvara_laatu_heikkeneminen', 0.807164192199707), ('maa_puhdistaminen', 0.7979235

70 [('ympäristönsuojelujärjestö', 0.768477201461792), ('luonnonsuojelu', 0.7486464977264404), ('lastensuojelujärjestö', 0.7465183734893799), ('luonnonsuojeluyhdistys', 0.7283951044082642), ('ympäristöjärjestö', 0.7239370942115784), ('luonnonsuojeluliitto', 0.7062997817993164), ('luonnonsuojeluohjelma', 0.691562294960022), ('luonnonsuojelualue', 0.6791955232620239), ('vapaaehtoinen_luonnonsuojelualue', 0.6709541082382202), ('luonnonsuojelija', 0.663583517074585)]

-----
 kaupunkialue
71 [('suurkaupunkialue', 0.8887217044830322), ('esikaupunkialue', 0.8630576729774475), ('pääkaupunkialue', 0.8457313179969788), ('kaupunki_viheralue', 0.7992988228797913), ('kaupunki', 0.7902138829231262), ('maaseutu_asuinalue', 0.772165060043335), ('elinympäristö_kaupunki', 0.771778404712677), ('kantakaupunki', 0.7639115452766418), ('kaupunki_keskus', 0.7524353265762329), ('kaupunki_kaatopaikka', 0.7500454187393188)]

-----
 maaseutu_haja_asutusalue
72 [('haja_asutusalue', 0.9508363008499146), ('haja', 0.8

88 [('vaikutus_ihminen', 0.859384298324585), ('moni_tekijä_aiheuttaa_saastuminen', 0.857061505317688), ('ihminen_aiheutua_tekijä', 0.8562391996383667), ('ihminen', 0.8349111676216125), ('ihminen_sairaus', 0.8319447040557861), ('ihminen_terveys', 0.8308453559875488), ('ihminen_altistuminen_saaste', 0.8267288208007812), ('ihminen_toiminta', 0.8265066742897034), ('ihminen_luonto_suhde', 0.8232206106185913), ('tauti_aiheuttaa_organismi', 0.8085373044013977)]

-----
 maataloustuotanto_säännöstely
89 [('säännöstely', 0.9002548456192017), ('maataloustuotanto', 0.7741248607635498), ('syntyvyys_säännöstely', 0.7687649130821228), ('säännöstellä', 0.7155200242996216), ('tuotanto_rajoittaminen', 0.7128808498382568), ('maatalous_suunnittelu', 0.7123914957046509), ('maatalous', 0.7100019454956055), ('maatalous_aiheuttaa_saastuminen', 0.6996424198150635), ('maatalous_vaikutus_ympäristö', 0.6984410881996155), ('maatalous_karjankasvatus', 0.6971657276153564)]

-----
 kaupunkirakentaminen
90 [('kaupunki

122 [('petrokemia', 0.9560694694519043), ('metallurginen_teollisuus', 0.8354130387306213), ('teollisuus', 0.8344964385032654), ('kemia_teollisuus', 0.794980525970459), ('petrokemikaali', 0.7943952083587646), ('sähkötekninen_teollisuus', 0.7819585800170898), ('kemianteollisuus', 0.781690239906311), ('teollinen', 0.7687314748764038), ('metalliteollisuus', 0.758101224899292), ('koneteollisuus_koneenrakennusteollisuus', 0.7547487616539001)]

-----
 eroosio
123 [('eroosio_valvonta', 0.8556067943572998), ('vesieroosio', 0.6879310607910156), ('tuulieroosio', 0.6800178289413452), ('maaperä_saastuminen', 0.6655179262161255), ('maaperä_vaurioituminen', 0.6631751656532288), ('maaperä_tiivistyminen', 0.6410550475120544), ('maaperä_saaste', 0.6315357089042664), ('pohjavesi_saastuminen', 0.6302781105041504), ('painuma_maa_vajoaminen', 0.6209445595741272), ('saastua_maaperä', 0.6167285442352295)]

-----
 maavesi
124 [('onkivesi', 0.7495019435882568), ('niinivesi', 0.705618679523468), ('haukivesi', 0.

...

4328 [('kotitalousjäte_kaatopaikka', 0.8201491832733154), ('jäte_kaatopaikka', 0.8115702271461487), ('teollisuusjäte', 0.8047448396682739), ('jäte', 0.7923968434333801), ('jäte_jäte', 0.7923968434333801), ('pakkausjäte', 0.7906913757324219), ('puutarhajäte', 0.7889371514320374), ('radioaktiivinen_jäte_kaatopaikka', 0.7864294052124023), ('puujäte', 0.7782248854637146), ('jäte_varastointi', 0.7763665318489075)]

-----
 viljelymaa_hoito
4329 [('maatila_hoito', 0.8854719400405884), ('rannikkoalue_hoito', 0.8645544052124023), ('ympäristö_hoito', 0.862350583076477), ('hoito', 0.8555054068565369), ('matkailutoiminta_hoito', 0.8291495442390442), ('hoito_terapia', 0.7989891767501831), ('metsä_kestävä_hoito_käyttö', 0.7915695309638977), ('viljelymaa', 0.7548471093177795), ('kasvi_terveydenhoito', 0.7533668279647827), ('hoito_hallinto_johtaminen_käsittely', 0.7326446771621704)]

-----
 kehityspolitiikka
4330 [('kehityspoliittinen', 0.8529765009880066), ('yrityspolitiikka', 0.7883507013320923), ('

4347 [('hiilijalanjälki', 0.6608065366744995), ('ekologinen_jalanjälki', 0.6449418067932129), ('jalanjälki', 0.6386212110519409), ('tuote_ympäristöjalanjälki_koskea_menetelmä', 0.6271490454673767), ('vesijalanjälki', 0.6204902529716492), ('organisaatio_ympäristöjalanjälki_koskea_menetelmä', 0.6099876761436462), ('ympäristöarvo', 0.5815840363502502), ('ympäristöhyöty', 0.5782803297042847), ('ympäristövaikutus', 0.5689496397972107), ('ympäristö', 0.5689140558242798)]

-----
 ympäristöhallinto
4348 [('ympäristökeskus', 0.7439578175544739), ('kunnallinen_ympäristöpolitiikka', 0.696919322013855), ('ympäristösuojelu', 0.6949918270111084), ('ympäristönsuojelu', 0.6821967363357544), ('ympäristöala_toimintaohjelma', 0.6800349950790405), ('ympäristövalvonta', 0.6721166372299194), ('vesiensuojelu', 0.6656008958816528), ('ympäristötoimi', 0.6649802327156067), ('ympäristöpalvelu', 0.663644552230835), ('ympäristöala', 0.6615504026412964)]

-----
 ympäristölainsäädäntö
4349 [('ympäristölainsäädäntö_t

4366 [('kevyt_polttoöljy', 0.9548677206039429), ('polttoöljy', 0.8933091163635254), ('raskas', 0.8195857405662537), ('raskas_tavara_kuljetus', 0.7611038684844971), ('polttoaine', 0.7481882572174072), ('kevyt_neste_erottaa', 0.7411075830459595), ('polttoaine_kulutus', 0.7105734944343567), ('polttoaine_koostumus', 0.7092623710632324), ('kevyt', 0.7088244557380676), ('hiili_muuttaminen_kaasu_hiili_kaasuttaminen', 0.707430899143219)]

-----
 lakijäätikkö
4367 [('jäätikkö', 0.6510297060012817), ('jäätikköjoki', 0.6219680905342102), ('jäätikkö_massatase', 0.5701513290405273), ('mannerjäätikkö', 0.5620290040969849), ('tasankoalue', 0.5447975397109985), ('litosfääri_kivikehä', 0.5444073677062988), ('skandi', 0.5212607383728027), ('kilpitulivuori', 0.5123791098594666), ('vuorijono', 0.5109609365463257), ('kraatteri', 0.5108669996261597)]

-----
 jää_häviäminen
4368 [('jää', 0.8472375869750977), ('häviäminen', 0.7739526033401489), ('vesi_vaarantaminen', 0.706885576248169), ('maa_sijaita_lähde_ai

4384 [('tuottajuus', 0.8442947864532471), ('käyttäjä', 0.7301133871078491), ('käyttäjä_etu', 0.6293768882751465), ('ihminen_toiminta', 0.6261741518974304), ('käyttäjähallinta', 0.6254127025604248), ('tuottaja', 0.6249582767486572), ('käyttö(', 0.622949481010437), ('tuottaja_vastuu', 0.6197419762611389), ('yhteiskunnallinen_prosessi', 0.6175514459609985), ('internet_palvelu_tarjoaja', 0.6124067306518555)]

-----
 uusiutuva_energialähde_koskea_direktiivi
4385 [('uusiutuva_luonnonvara', 0.8344025015830994), ('uusiutua_energianlähde', 0.8248893022537231), ('vesi_laatu_koskea_direktiivi', 0.823694109916687), ('maatalous_koskea_ympäristölainsäädäntö', 0.8178622722625732), ('päästö_koskea_lainsäädäntö', 0.8172499537467957), ('uusiutuva', 0.8169450163841248), ('uusiutua', 0.8119989633560181), ('geeniteknologia_koskea_lainsäädäntö', 0.8006447553634644), ('direktiivi_saattaminen_osa_kansallinen_lainsäädäntö', 0.7990390658378601), ('vesivara_koskea_lainsäädäntö', 0.7988169193267822)]

-----
 vesi

4403 [('maatalous_koskea_ympäristölainsäädäntö', 0.8272231221199036), ('maatalous', 0.8140537738800049), ('terveys_liittyä_biotekniikka', 0.8118231296539307), ('maatalous_vaikutus_ympäristö', 0.8103899955749512), ('maatalous_rakenne', 0.8062611818313599), ('maatalous_saada_elintarvike', 0.8036503195762634), ('maatalous_suunnittelu', 0.7956839203834534), ('ympäristö_liittyä_sairaus', 0.7949222326278687), ('ympäristö_liittyä_vastuu', 0.7802509069442749), ('teollisuus_vaikutus_ympäristö', 0.771837592124939)]

-----
 ekologinen_kehitys
4404 [('ekologinen', 0.8843197822570801), ('biologinen_kehitys', 0.8831534385681152), ('ekologinen_sopeutuminen', 0.881516695022583), ('ekologinen_kokonaisuus_tutkimus', 0.880134642124176), ('ympäristö_kehitys', 0.879776656627655), ('taloudellinen_kehitys', 0.8675627112388611), ('ekologinen_sivilisaatio', 0.8634112477302551), ('ekologinen_tasapaino', 0.8561384677886963), ('ekologinen_jalanjälki', 0.8552703857421875), ('ekologinen_yhteisö', 0.8545111417770386